In [0]:
# create a notebook for LSTM prediction model 

# 1. Load the chosen dataset and try to see the vocab size 
# 2. Determine a MAX_VOCAB_SIZE incase you observe a vocab dict that is too large (choose the top/most frequent MAX_VOCAB_SIZE entries / Curse of dimensionality)
# 3. Determine a MAX_SEQUENCE LENGTH to vectorize for each review (Note: From splitter usually reviews will be limited to 400 + summary )

# DO NOT EDIT/ DELETE THIS BLOCK;



In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [26]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab or local 
%cd /content/drive/My\ Drive/SNLP\ Project
# %cd Source/repos/Sentiment-Analysis-using-Deep-Learning

/content/drive/My Drive/SNLP Project


In [27]:
# list content of drive - verify you are where you are supposed to be
%ls

 CleantData_Apr-03-2020_01-31.zip        'Project Ideas.gdoc'
 Data_Balanced_20000_Apr-03-2020_06-52/  'Project Proposal.gdoc'
 Data_Balanced_2000_Apr-03-2020_06-46/   'Report MetaData'/
 Electronics_5.json.gz


In [28]:
# load your choice of dataset here . Specify paths as folder_datestring/file_datestring.zip
Train_ZipCSVFileName = 'Data_Balanced_2000_Apr-03-2020_06-46/Train_2000_Apr-03-2020_06-46.zip'
Test_ZipCSVFileName = 'Data_Balanced_2000_Apr-03-2020_06-46/Test_2000_Apr-03-2020_06-46.zip'

import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         10000 non-null  float64
 1   reviewTime      10000 non-null  object 
 2   reviewerID      10000 non-null  object 
 3   asin            10000 non-null  object 
 4   reviewText      10000 non-null  object 
 5   summary         10000 non-null  object 
 6   unixReviewTime  10000 non-null  int64  
 7   reviewText_len  10000 non-null  int64  
 8   summary_len     10000 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 703.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         1000 non-null   float64
 1   reviewTime      1000 non-null   object 
 2   reviewerID      1000 non-null   object 
 3  

In [29]:
df_train['reviewText_len'].describe()
# Since the mean average review size is around 145 chars and max is 400, I can safely set the max [summary + review] Text Limit to 400 



count    10000.000000
mean       144.724600
std        108.495885
min          1.000000
25%         47.000000
50%        126.000000
75%        222.000000
max        399.000000
Name: reviewText_len, dtype: float64

In [0]:
# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 200

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [66]:
tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['reviewText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 8707 unique tokens.


In [67]:
X = tokenizer.texts_to_sequences(df_train['reviewText'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Shape of data tensor: (10000, 250)


In [68]:
Y = pd.get_dummies(df_train['overall']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (10000, 5)


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=X.shape[1]))
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(200))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10

history = model.fit(X, Y, epochs=epochs)

In [72]:


X_test = tokenizer.texts_to_sequences(df_test['reviewText'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['overall']).values
print('Shape of label tensor:', Y_Test.shape)

accr = model.evaluate(X_test,Y_Test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Shape of data tensor: (1000, 250)
Shape of label tensor: (1000, 5)
32/32 [==============================] - 6s 196ms/step - loss: 2.4391 - accuracy: 0.4340
Test set
  Loss: 2.439
  Accuracy: 0.434
